In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Dataset**

In [2]:
df = pd.read_csv('../input/stocknews/Combined_News_DJIA.csv', encoding = "ISO-8859-1")
df.head()

**Split Train and Test Set**

In [3]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [4]:
data = train.iloc[:, 2:27]
data.replace("[^a-zA-Z]", " ", regex = True, inplace = True)

colNames = [i for i in range(25)]
newIndex = [str(i) for i in colNames]
data.columns = newIndex
data.head()

In [5]:
for index in newIndex:
    data[index] = data[index].str.lower()
    data[index]
data.head()

In [6]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
for index in newIndex:
    for st in data[index]:
        if type(st) != 'str':
            continue
        words = st.split(" ");
        for word in words:
            word = lem.lemmatize(word)
        newWords = " ".join(words)
        st = newWords
data.head()

In [7]:
" ".join(str(x) for x in data.iloc[1, 0:25])

In [8]:
headlines = []
for row in range(0, len(data.index)):
    headlines.append(" ".join(str(x) for x in data.iloc[row, 0:25]))
headlines[0]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [10]:
countvector = CountVectorizer(ngram_range = (2, 2))
traindataset = countvector.fit_transform(headlines)

In [11]:
randomClassifier = RandomForestClassifier(n_estimators = 250, criterion = 'entropy')
randomClassifier.fit(traindataset, train['Label'])

In [12]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomClassifier.predict(test_dataset)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [14]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)

In [15]:
score = accuracy_score(test['Label'], predictions)
print(score)

In [16]:
report = classification_report(test['Label'], predictions)
print(report)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [18]:
tfidfvector = TfidfVectorizer(ngram_range = (2, 2))
traindataset = tfidfvector.fit_transform(headlines)

In [19]:
randomClassifier = RandomForestClassifier(n_estimators = 250, criterion = 'entropy')
randomClassifier.fit(traindataset, train['Label'])

In [20]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
test_dataset = tfidfvector.transform(test_transform)
predictions = randomClassifier.predict(test_dataset)

In [21]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)

In [22]:
score = accuracy_score(test['Label'], predictions)
print(score)

In [23]:
report = classification_report(test['Label'], predictions)
print(report)

In [24]:
from sklearn.naive_bayes import MultinomialNB

In [25]:
naive = MultinomialNB()
naive.fit(traindataset, train['Label'])

In [33]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
test_dataset = tfidfvector.transform(test_transform)
predictions = naive.predict(test_dataset)

In [27]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)

In [28]:
score = accuracy_score(test['Label'], predictions)
print(score)

In [29]:
report = classification_report(test['Label'], predictions)
print(report)